In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from time import sleep, time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime, timedelta

In [3]:
class DataColector():
    def __init__(self):
        self.options = webdriver.FirefoxOptions()
        self.options.headless = True
        self.options.add_argument('--headless')
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.chrome = webdriver.Firefox(executable_path=r'geckodriver.exe', options=self.options)
    
    def acessa(self, url):
        self.chrome.get(url)
    
    def coleta_html(self):
        html = soup(self.chrome.page_source, 'html.parser')
        return html

    def aguardar_por(self, xpath):
        WebDriverWait(self.chrome, 20).until(
            EC.visibility_of_element_located(
                (By.XPATH, xpath)))

    def sair(self):
        self.chrome.quit()

In [121]:
class Driver(DataColector):

    def scroll(self):
        scroll_pause_time = 3

        # Get scroll height
        last_height = self.chrome.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            self.chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            sleep(scroll_pause_time)

            # Calculate new scroll height and compare with last scroll height
            new_height = self.chrome.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # If heights are the same it will exit the function
                break
            last_height = new_height
    
    def clicla_carregar_mais(self, qnts_vezes):
        try:
            for n in range(32, 32 + (qnts_vezes * 10), 10):
                self.aguardar_por(f'/html/body/div[6]/div[4]/div/div/div[1]/div/div[{n}]/a')
                element = self.chrome.find_element_by_xpath(f'/html/body/div[6]/div[4]/div/div/div[1]/div/div[{n}]/a')
                self.chrome.execute_script('arguments[0].click();', element)
        except Exception as e:
            print(f'Erro ao clicar em carregar mais: {e}')

    def listar_noticias(self, cod_news):
        try:
            all_articles = []
            articles = cod_news.findAll('h3')
            for article in articles:
                news_dict = dict()
                title = article.a
                news_dict['titulo'] = 'None' if not title else title.text
                news_dict['portal'] = 'SoNoticiaBoa'
                news_dict['sentiment'] = 'pos'
                all_articles.append(news_dict)
            print(len(all_articles))
            return all_articles
        except Exception as e:
            print(f'Erro ao tentar listar noticias:', e)

if __name__ == '__main__':
    browser = Driver()
    browser.acessa('https://www.sonoticiaboa.com.br/mundo/')
    browser.scroll()  # Cada clique, +10 noticias são exibidas
    browser.clicla_carregar_mais(200)
    html = browser.coleta_html()
    browser.sair()
    article_news = browser.listar_noticias(html)

Erro ao clicar em carregar mais: Message: 

917


In [122]:
df_articles = pd.DataFrame(article_news)

In [123]:
df_articles.head()

,titulo,portal,sentiment
0,Última Ceia em altíssima resolução mostra deta...,SoNoticiaBoa,pos
1,Cientistas do Canadá criam máscara que desativ...,SoNoticiaBoa,pos
2,É hoje! Júpiter aparecerá mais brilhante e pod...,SoNoticiaBoa,pos
3,Sobrinha ensina tio analfabeto a ler e escreve...,SoNoticiaBoa,pos
4,Lewis Hamilton faz 2 protestos contra racismo ...,SoNoticiaBoa,pos


In [124]:
# Removing all "None's" values from DataFrame 
for index, row in df_articles.iterrows():
    if 'None' in row['titulo']:
        df_articles = df_articles.drop(index, axis=0)

In [125]:
df_articles.to_csv('snb_positive_news.csv')

In [102]:
class DriverVeja(DataColector):
   
    def clicla_carregar_mais(self, qnts_vezes):
        try:
            for n in range(21, 21 + (qnts_vezes * 20), 20):
                self.aguardar_por(f'/html/body/div[4]/section/div/div[1]/section/div/div[{n}]/span/button')
                element = self.chrome.find_element_by_xpath(f'/html/body/div[4]/section/div/div[1]/section/div/div[{n}]/span/button')
                element.click()
        except Exception as e:
            print(f'Erro ao clicar em carregar mais: {e}')

    def listar_noticias(self, cod_news):
        try:
            all_articles = []
            articles = cod_news.findAll('h2')
            for article in articles:
                news_dict = dict()
                title = article
                news_dict['titulo'] = 'None' if not title else title.text
                news_dict['portal'] = 'Veja'
                news_dict['sentiment'] = 'neg'
                all_articles.append(news_dict)
            print(len(all_articles))
            return all_articles
        except Exception as e:
            print(f'Erro ao tentar listar noticias:', e)

if __name__ == '__main__':
    browser = DriverVeja()
    browser.acessa('https://veja.abril.com.br/noticias-sobre/tragedia/')
    browser.clicla_carregar_mais(200)
    html = browser.coleta_html()
    browser.sair()
    article_news = browser.listar_noticias(html)

Erro ao clicar em carregar mais: Message: 

1123


In [103]:
df_articles = pd.DataFrame(article_news)

In [109]:
df_articles

,titulo,portal,sentiment
0,Sobe para 41 o número de mortos na Baixada San...,Veja,neg
1,"Vítimas de Suzano, Brumadinho e da Boate Kiss:...",Veja,neg
2,A revolta da mãe de vítima do Ninho do Urubu u...,Veja,neg
3,Ninho do Urubu: famílias ainda aguardam indeni...,Veja,neg
4,Artigo: ‘O que fazer para evitar novas tragédi...,Veja,neg
...,...,...,...
1118,Tragédia aérea – Acreditem: Lula supera Lula n...,Veja,neg
1119,Bombeiros resgatam 14 corpos de destroços de a...,Veja,neg
1120,Avião cai e mata 11 em Trancoso. Banqueiro est...,Veja,neg
1121,Riscos de deslizamentos continuam,Veja,neg


In [111]:
class DriverExame(DataColector):
   
    def clicla_carregar_mais(self, qnts_vezes):
        try:
            for n in range(qnts_vezes):
                self.aguardar_por('/html/body/div[4]/div/div[1]/ul/div[3]/span/button')
                element = self.chrome.find_element_by_xpath('/html/body/div[4]/div/div[1]/ul/div[3]/span/button')
                element.click()
        except Exception as e:
            print(f'Erro ao clicar em carregar mais: {e}')

    def listar_noticias(self, cod_news):
        try:
            all_articles = []
            articles = cod_news.findAll('span', {'class' : 'list-item-title'})
            for article in articles:
                news_dict = dict()
                title = article.a
                news_dict['titulo'] = 'None' if not title else title.text
                news_dict['portal'] = 'Exame'
                news_dict['sentiment'] = 'neg'
                all_articles.append(news_dict)
            print(len(all_articles))
            return all_articles
        except Exception as e:
            print(f'Erro ao tentar listar noticias:', e)

if __name__ == '__main__':
    browser = DriverExame()
    browser.acessa('https://exame.com/noticias-sobre/mortes/')
    browser.clicla_carregar_mais(200)
    html = browser.coleta_html()
    browser.sair()
    article_news = browser.listar_noticias(html) 

2000


In [112]:
df_articles = pd.DataFrame(article_news)

In [113]:
df_articles

,titulo,portal,sentiment
0,Negros são 75% dos mortos pela polícia no Bras...,Exame,neg
1,Cientistas descobrem anticorpos potentes contr...,Exame,neg
2,Maioria das grávidas que morreram de covid-19 ...,Exame,neg
3,Morre o ex-presidente da Câmara Severino Caval...,Exame,neg
4,"Pela primeira vez, estado de SP supera 15 mil ...",Exame,neg
...,...,...,...
1995,Choque entre caminhonete e caminhão deixa 12 m...,Exame,neg
1996,Chacina em Porto Alegre deixa sete mortos e um...,Exame,neg
1997,Naufrágio em lago do Missouri deixa 17 mortos,Exame,neg
1998,Sobe para 125 número de mortos por erupção na ...,Exame,neg


In [114]:
df_articles.to_csv('exame_negative_news.csv')

In [131]:
df_veja = pd.read_csv('dataset/veja_negative_news.csv')
df_exame = pd.read_csv('dataset/exame_negative_news.csv')
df_g1 = pd.read_csv('dataset/g1_positive_news.csv')
df_snb_br = pd.read_csv('dataset/snb_positive_news.csv')
df_snb_world = pd.read_csv('dataset/snb_positive_news2.csv')

In [133]:
df_complied = pd.concat([df_veja, df_exame, df_g1, df_snb_br, df_snb_world])

In [134]:
df_complied

,Unnamed: 0,titulo,portal,sentiment
0,0,Sobe para 41 o número de mortos na Baixada San...,Veja,neg
1,1,"Vítimas de Suzano, Brumadinho e da Boate Kiss:...",Veja,neg
2,2,A revolta da mãe de vítima do Ninho do Urubu u...,Veja,neg
3,3,Ninho do Urubu: famílias ainda aguardam indeni...,Veja,neg
4,4,Artigo: ‘O que fazer para evitar novas tragédi...,Veja,neg
...,...,...,...,...
910,910,Vacina contra câncer está pronta pra testes em...,SoNoticiaBoa,pos
911,911,Empresa abre 10 mil vagas em home office em pl...,SoNoticiaBoa,pos
912,912,Rússia promete lançar vacina contra Covid no m...,SoNoticiaBoa,pos
913,913,Máscaras transparentes deixam respirar com sor...,SoNoticiaBoa,pos


In [135]:
df_complied.to_csv('dataset/complied_sentiment_news.csv')

In [136]:
df_complied.sentiment.value_counts()

pos    3346
neg    3123
Name: sentiment, dtype: int64